In [1]:
import pandas as pd

df = pd.read_csv("gpt.csv")
df.head()

,Punctuation Type,Sentence (Wrote),Sentence (Meant),GPT-5
0,Comma,Chanting the choir raised the volume as the ce...,"Chanting, the choir raised the volume as the c...",मंडळातील गायन करताना गायकांचा आवाज वाढवला गेला...
1,Comma,A six-month-old calf was submitted for examina...,A six-month-old calf was submitted for examina...,"सहा महिन्यांचा बछडा तपासणीसाठी सादर केला गेला,..."
2,Comma,Planning authorities should provide alternativ...,Planning authorities should provide alternativ...,नियोजन प्राधिकरणांनी अशा लघु व्यवसायांसाठी पर्...
3,Comma,As the machine develops the forms we use to re...,"As the machine develops, the forms we use to r...","मशीन जसे स्वरूप विकसित करते, तसेच आपण भूतपूर्व..."
4,Comma,"As mentioned, first impressions can be mislead...","As mentioned first, impressions can be mislead...","जसे म्हटले, पहिले छाप चुकीचे ठरू शकतात."


In [2]:
translations = [sent.strip() for sent in df['GPT-5']]
translations

['मंडळातील गायन करताना गायकांचा आवाज वाढवला गेला, जेव्हा विधीकर्ता प्रार्थना उच्चारत होता.',
 'सहा महिन्यांचा बछडा तपासणीसाठी सादर केला गेला, ज्यामध्ये जन्मानंतर लगेचपासूनच सर्व चार पायांमध्ये लंगडेपणा होता.',
 'नियोजन प्राधिकरणांनी अशा लघु व्यवसायांसाठी पर्यायी ठिकाणे उपलब्ध करून द्यावी, जे निवासी भागात असू शकतात किंवा त्रासदायक ठरू शकतात.',
 'मशीन जसे स्वरूप विकसित करते, तसेच आपण भूतपूर्व प्रकल्पांमधून डेटा नोंदवण्यासाठी वापरलेली पद्धत सुधारली जाईल.',
 'जसे म्हटले, पहिले छाप चुकीचे ठरू शकतात.',
 'स्वच्छ असेंब्ली मिळवण्यासाठी असेंब्ली चालवण्यापूर्वी असेंबल केलेली ‘इक्वल्स’ टेबल लोड करा.',
 'कार्यवाही करणारे व्यक्ती ठरलेल्या तारखांपासून मोठ्या फरकाबाबत माहिती देण्यात विलंब करतात. त्यामुळे वेळेवर कारवाई करता येत नाही.',
 'हे ग्लायकन्स प्रोटिनवर नीट हस्तांतरित होत नाहीत, ज्यामुळे ग्लायकॉसिलेशन सिक्वॉन्स रिक्त राहतात.',
 'X हा मायग्रेनसाठी प्रभावी तीव्र, तोंडाने दिला जाणारा उपचार आहे, ज्याची क्रियाशीलता वेगाने सुरू होते.',
 'माझ्या तज्ञ मतानुसार कोणताही वृत्तपत्र पूर्णपणे पक्षपाती नाही.',

In [3]:
from datasets import load_dataset
raw_datasets = load_dataset("thenlpresearcher/test_data_marathi")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
src_sentences = raw_datasets['test']["sent_written"]
ref_gt     = raw_datasets['test']["gt_marathi"]
ref_gem    = raw_datasets['test']["gemini_out"]
ref_cfilt  = raw_datasets['test']["cfilt_out"]

In [5]:
mode = "gpt"

In [6]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"shalaka_{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to shalaka_{mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=references, char_order=6, word_order=2,  beta=2)["score"]
    return bleu_score, chrf_score, chrf2_score

bleu_score, chrf_score, chrf2_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"shalaka_punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to shalaka_punct_{mode}_indictrans2_eval_metrics.txt")

2025-11-25 10:11:11.248127: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 10:11:11.315519: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 10:11:13.684670: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


✔ Saved predictions to shalaka_gpt_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 28.62, chrF++: 58.09, chrF2++: 54.55
GT Marathi → BLEU: 14.59
Gemini    → BLEU: 18.69
CFILT     → BLEU: 15.60

🎯 BEST REFERENCE = Gemini (by highest BLEU)
Metrics written to shalaka_punct_gpt_indictrans2_eval_metrics.txt
